# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [6]:
# imports
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [7]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
MODEL_GEMMA = 'google/gemma-3-12b'

In [8]:
# set up environment

In [9]:
# here is the question; type over this to ask something new

question = """
Пожалуйста, объясните, что делает этот код и почему:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# # Get gemma-3-12b to answer
# gemma = OpenAI(base_url='http://localhost:1234/v1', api_key='google/gemma-3-12b')
# response = gemma.chat.completions.create(
#     model=MODEL_GEMMA,
#     messages=[{"role": "user", "content": question}],
# )

# display(Markdown(response.choices[0].message.content))

# Get gemma-3-12b to answer, with streaming
gemma = OpenAI(base_url='http://localhost:1234/v1', api_key='google/gemma-3-12b')
stream = gemma.chat.completions.create(
    model=MODEL_GEMMA,
    messages=[
        {"role": "user", "content": question}
        ],
    stream=True
)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

Этот код использует генераторное выражение с `yield from` для извлечения авторов из списка книг, но только тех, у которых есть информация об авторе. Давайте разберем его по частям:

**1. `books`:**

Предполагается, что `books` - это список словарей (или другой итерируемый объект), где каждый словарь представляет книгу.  Каждый словарь книги содержит ключи и значения, описывающие эту книгу.

**2. `book.get("author")`:**

*   `book`: Это переменная, представляющая текущую книгу в списке `books`, которую обрабатывает цикл.
*   `.get("author")`: Этот метод словаря пытается получить значение по ключу "author".  Важно отметить, что `.get()` - это безопасный способ доступа к ключам словаря. Если ключ "author" отсутствует в словаре `book`, он вернет `None` (или другое указанное значение по умолчанию, если оно предоставлено как второй аргумент для `.get()`). Это предотвращает ошибку `KeyError`.

**3. `for book in books if book.get("author")`:**

Это генераторное выражение с фильтрацией.  Оно итерируется по списку `books`, но пропускает только те книги, у которых есть ключ "author" (т.е., `book.get("author")` не возвращает `None`).  Условие `if book.get("author")` проверяет, что значение, возвращаемое `book.get("author")`, является истинным (не `None`, пустая строка, 0 и т.д.).

**4. `{... for ... if ...}`:**

Это синтаксис генераторного выражения.  Он создает итератор, который генерирует значения по требованию. В данном случае он генерирует авторов для книг, у которых есть информация об авторе.

**5. `yield from`:**

*   `yield`: Это ключевое слово, которое делает функцию (или генераторное выражение) генератором. Генераторы позволяют создавать итераторы, которые выдают значения по одному, не храня все результаты в памяти сразу.
*   `from`:  Ключевое слово `yield from` используется для "делегирования" генерации другому итератору. В данном случае, оно делегирует генерацию авторов из генераторного выражения `{book.get("author") for book in books if book.get("author")}` вызывающему коду.  Это означает, что `yield from` эффективно передает все значения, генерируемые внутренним генератором, как если бы они были сгенерированы непосредственно внешним генератором.

**В итоге:**

Этот код создает генератор, который выдает имена авторов из списка книг, но только для тех книг, у которых есть информация об авторе.  Он делает это эффективно, не храня все имена авторов в памяти одновременно.

**Почему используется `yield from`?**

Использование `yield from` имеет несколько преимуществ:

*   **Эффективность:**  Вместо того чтобы собирать все имена авторов в список и затем выдавать их по одному, `yield from` позволяет выдавать значения непосредственно из внутреннего генератора. Это экономит память, особенно если список книг очень большой.
*   **Простота:**  `yield from` делает код более читаемым и понятным, поскольку он явно показывает, что мы делегируем генерацию другому итератору.

**Пример:**

Предположим, `books = [ {"title": "Книга 1", "author": "Толстой"}, {"title": "Книга 2"}, {"title": "Книга 3", "author": "Достоевский"} ]`

Когда вы итерируетесь по результату этого кода:

*   Первая книга имеет автора "Толстой". `yield from` выдаст "Толстой".
*   Вторая книга не имеет автора. Она будет пропущена из-за условия `if book.get("author")`.
*   Третья книга имеет автора "Достоевский".  `yield from` выдаст "Достоевский".

Таким образом, код вернет итератор, который последовательно выдает значения "Толстой" и "Достоевский".

**В заключение:** Этот код представляет собой компактный и эффективный способ извлечения данных из списка словарей с использованием генераторов и `yield from`. Он демонстрирует хорошие практики программирования на Python, такие как использование `.get()` для безопасного доступа к ключам словаря и использование генераторных выражений для создания итераторов.


In [11]:
# Get Llama 3.2 to answer
llama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
response = llama.chat.completions.create(
    model=MODEL_LLAMA,
    messages=[{"role": "user", "content": question}],
)

display(Markdown(response.choices[0].message.content))

Этот код utiliza функцию `yield from` из Python 3.x для генерации последовательности значений из других генераторов.

В данном случае мы имеем дело с фрагментом хукера на example.com за исключением определенных book в "пустом" книгах. Это будет иметь в виду, сколько и какие книги на простой книге: 

`yield from {book.get("author") for book in books if not book or book.get('in_book', False)}`

Чтобы это Understanding было более четко, давайте разобъясним его составные части:

- `books`: Это генератор или массив книг, из которых нам нужны данные.
  
  - `yield from [ ... ]` — это функция для генерации последовательности значений из других génераторов. В данном случае она означает получить последовательность значений книги по ключ «author» для всех книг.

- `{book.get("author") for book in books if book.get("author")}` это генератор последовательности книг, где значения из ключа "author" представлены значениями в словаре book.
- `yield from` означает: если эта функция не может сжать заданный генератор или генератор уже сжат.